In [2]:
import pandas as pd
import re
from joblib import Parallel, delayed

In [35]:

def clima(m,anio):
    
    '''Funcion que saca las medias de los datos de temperatura y precipitaciones en España
        La función además de descargar los ficheros quel tiempo configura el dataframe'''
    
    tiempo=pd.DataFrame(columns=['fecha','temp_max', 'temp_min', 'temp_media','precipitacion'] )
    
       
    lst=['01','02','03','04','05','06','07','08','09','10','11','12','13','14','15'
         '16','17','18','19','20','21','22','23','24','25','26','27','28','29','30','31']
    
    for d in lst:
        try:
            print('carga')
            df=pd.read_excel(f'../tiempo/Aemet{anio}-{m}/Aemet{anio}-{m}-{d}.xls')
        except:
            print('No carga error')
            continue

        df.dropna(inplace=True)
                
        df.columns=df.loc[3]
                
        df.drop([3],axis=0, inplace=True)
                
        df['Temperatura máxima (ºC)']=df['Temperatura máxima (ºC)'].apply(lambda x: re.findall('\d+.\d+',x)[0])
        df['Temperatura mínima (ºC)']=df['Temperatura mínima (ºC)'].apply(lambda x: re.findall('\d+.\d+',x)[0])

        def to_int(val):
            """ Reconoce valores numericos y los transforma a enteros.
            """
            try:
                value = int(float(val))
            except ValueError:
                value = ""
            return value

        df['Temperatura máxima (ºC)']=df['Temperatura máxima (ºC)'].apply(to_int)
        df['Temperatura mínima (ºC)']=df['Temperatura mínima (ºC)'].apply(to_int)
                
        fecha='2017-01-01'
        temp_max=round(df['Temperatura máxima (ºC)'].mean(),2)
        temp_min=round(df['Temperatura mínima (ºC)'].mean(),2)
        temp_media=round(df['Temperatura media (ºC)'].mean(),2)
        precipitacion=round(df['Precipitación 00-24h (mm)'].mean(),2)

        tiempo = tiempo.append({'fecha': f'{anio}-{m}-{d}', 'temp_max':temp_max, 'temp_min':temp_min,
                        'temp_media':temp_media,'precipitacion':precipitacion}, ignore_index=True)
    
        
    return tiempo

    

In [42]:
anio=['2017','2018','2019','2020','2021','2022']
mes=['01','02','03','04','05','06','07','08','09','10','11','12']

In [43]:
#Descargamos los archivos sobre el clima
final=[]
for a in anio:
    lst=Parallel(n_jobs=-1, verbose=True)(delayed(clima)(e,a) for e in mes)
    final.append(pd.concat(lst))
    
    

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=-1)]: Done  12 out of  12 | elapsed:    4.5s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=-1)]: Done  12 out of  12 | elapsed:    5.4s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=-1)]: Done  12 out of  12 | elapsed:    4.5s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=-1)]: Done  12 out of  12 | elapsed:    4.4s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=-1)]: Done  12 out of  12 | elapsed:    4.4s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=-1)]: Done  12 out of  12 | elapsed:    4.5s finished


In [45]:
#Concatenamos todos los dataframe
tabla_tiempo=pd.concat(final)
tabla_tiempo

,fecha,temp_max,temp_min,temp_media,precipitacion
0,2017-01-01,10.11,4.36,5.48,0.03
1,2017-01-02,11.05,4.36,6.81,0.19
2,2017-01-03,13.13,4.55,8.61,0.46
3,2017-01-04,13.68,5.14,9.3,0.06
4,2017-01-05,12.9,4.36,8.4,0.07
...,...,...,...,...,...
23,2022-11-26,16.46,4.99,10.93,0.02
24,2022-11-27,16.33,4.93,10.78,1.84
25,2022-11-28,13.9,6.07,10.36,2.82
26,2022-11-29,13.8,5.28,9.82,1.1


In [46]:
#Guadamos el archivo
tabla_tiempo.to_csv('../data/tabla_tiempo.csv', index=False,index_label=False)

In [28]:
tabla_tiempo=pd.read_csv('../data/tabla_tiempo.csv')

In [ ]:
tabla_tiempo.describe(include='all').T

In [10]:
tabla_tiempo.head()

,fecha,temp_max,temp_min,temp_media,precipitacion
0,2017-01-01,"10,11","4,36","5,48","0,03"
1,2017-01-02,"11,05","4,36","6,81","0,19"
2,2017-01-03,"13,13","4,55","8,61","0,46"
3,2017-01-04,"13,68","5,14","9,3","0,06"
4,2017-01-05,"12,9","4,36","8,4","0,07"


In [29]:
listas=pd.read_csv('../data/lista_definitiva.csv')

In [30]:

prueba=pd.merge(listas,tabla_tiempo, left_on="fecha", right_on='fecha', how='left')

In [14]:
prueba.head()

,fecha,uri,artist_names,track_name,source,position,previous_rank,dif,peak_rank,days_on_chart,streams,estilo,temp_max,temp_min,temp_media,precipitacion
0,2017-01-01,spotify:track:5aAx2yezTd8zXrkmtKl66Z,"the weeknd, daft punk",Starboy,Universal Republic Records,1,-1,199,1,1,3135625,pop,"10,11","4,36","5,48","0,03"
1,2017-01-01,spotify:track:7BKLCZ1jbUBVqRi2FVlTVw,"the chainsmokers, halsey",Closer,Disruptor Records/Columbia,2,-1,198,2,1,3015525,pop,"10,11","4,36","5,48","0,03"
2,2017-01-01,spotify:track:4pdPtRcBmOSQDlJ3Fk945m,"dj snake, justin bieber",Let Me Love You,DJ Snake Def Jam,3,-1,197,3,1,2545384,trap,"10,11","4,36","5,48","0,03"
3,2017-01-01,spotify:track:5knuzwU65gJK7IF5yJsuaW,"clean bandit, sean paul, anne-marie",Rockabye (feat. Sean Paul & Anne-Marie),Atlantic Records UK,4,-1,196,4,1,2356604,pop,"10,11","4,36","5,48","0,03"
4,2017-01-01,spotify:track:1xznGGDReH1oQq0xzbwXa3,"drake, wizkid, kyla",One Dance,Cash Money Records/Young Money Ent./Universal ...,5,-1,195,5,1,2259887,trap,"10,11","4,36","5,48","0,03"


In [31]:
eventos=pd.read_excel('../data/Eventos.xlsx')

In [ ]:
eventos.info(), listas.info()

In [35]:

#pd.to_datetime('10/11/12', format='%d/%y/%m')

prueba['fecha'] = pd.to_datetime(prueba.fecha)

In [24]:
listas.head()

,fecha,uri,artist_names,track_name,source,position,previous_rank,dif,peak_rank,days_on_chart,streams,estilo
0,2017-01-01,spotify:track:5aAx2yezTd8zXrkmtKl66Z,"the weeknd, daft punk",Starboy,Universal Republic Records,1,-1,199,1,1,3135625,pop
1,2017-01-01,spotify:track:7BKLCZ1jbUBVqRi2FVlTVw,"the chainsmokers, halsey",Closer,Disruptor Records/Columbia,2,-1,198,2,1,3015525,pop
2,2017-01-01,spotify:track:4pdPtRcBmOSQDlJ3Fk945m,"dj snake, justin bieber",Let Me Love You,DJ Snake Def Jam,3,-1,197,3,1,2545384,trap
3,2017-01-01,spotify:track:5knuzwU65gJK7IF5yJsuaW,"clean bandit, sean paul, anne-marie",Rockabye (feat. Sean Paul & Anne-Marie),Atlantic Records UK,4,-1,196,4,1,2356604,pop
4,2017-01-01,spotify:track:1xznGGDReH1oQq0xzbwXa3,"drake, wizkid, kyla",One Dance,Cash Money Records/Young Money Ent./Universal ...,5,-1,195,5,1,2259887,trap


In [36]:
prueba2=pd.merge(prueba,eventos, left_on="fecha", right_on='dates', how='left')

In [37]:
prueba2.head()

,fecha,uri,artist_names,track_name,source,position,previous_rank,dif,peak_rank,days_on_chart,...,estilo,temp_max,temp_min,temp_media,precipitacion,dates,event,type,region,fin_de_semana
0,2017-01-01,spotify:track:5aAx2yezTd8zXrkmtKl66Z,"the weeknd, daft punk",Starboy,Universal Republic Records,1,-1,199,1,1,...,pop,"10,11","4,36","5,48","0,03",2017-01-01,NaN,NaN,NaN,7
1,2017-01-01,spotify:track:7BKLCZ1jbUBVqRi2FVlTVw,"the chainsmokers, halsey",Closer,Disruptor Records/Columbia,2,-1,198,2,1,...,pop,"10,11","4,36","5,48","0,03",2017-01-01,NaN,NaN,NaN,7
2,2017-01-01,spotify:track:4pdPtRcBmOSQDlJ3Fk945m,"dj snake, justin bieber",Let Me Love You,DJ Snake Def Jam,3,-1,197,3,1,...,trap,"10,11","4,36","5,48","0,03",2017-01-01,NaN,NaN,NaN,7
3,2017-01-01,spotify:track:5knuzwU65gJK7IF5yJsuaW,"clean bandit, sean paul, anne-marie",Rockabye (feat. Sean Paul & Anne-Marie),Atlantic Records UK,4,-1,196,4,1,...,pop,"10,11","4,36","5,48","0,03",2017-01-01,NaN,NaN,NaN,7
4,2017-01-01,spotify:track:1xznGGDReH1oQq0xzbwXa3,"drake, wizkid, kyla",One Dance,Cash Money Records/Young Money Ent./Universal ...,5,-1,195,5,1,...,trap,"10,11","4,36","5,48","0,03",2017-01-01,NaN,NaN,NaN,7


In [27]:
prueba2.to_csv('../data/all_lista.csv', index=False,index_label=False)